# Seven Segment Search

In [89]:
with open("08_seven_segment_search.txt", "r") as f:
    data = f.readlines()

In [90]:
testdata = """
be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce
""".splitlines()[1:]

In [91]:
def parse_input(input_data):
    line_data = []
    for line in input_data:
        front_string, back_string = line.split("|")
        signal_patterns = [set(signal) for signal in front_string.split()]
        output_pattern = [set(signal) for signal in back_string.split()]
        line_data.append((signal_patterns, output_pattern))

    return line_data

In [92]:
from collections import defaultdict

## Part 1
1, 7, 4 and 8 are easily identifiable by their signal length.

In [93]:
def get_count_of_easily_identifiable_digits(input_data):
    signal_length_to_digit = {
        2: 1,
        3: 7, 
        4: 4,
        7: 8,
    }
    
    digit_occurrences = defaultdict(int)
    
    for _, output_pattern in parse_input(input_data):
        for signal_set in output_pattern:
            segment_count = len(signal_set)
            if segment_count in signal_length_to_digit:
                digit = signal_length_to_digit[segment_count]
                digit_occurrences[digit] += 1
    
    total_count = sum(digit_occurrences.values())
    return total_count


In [94]:
get_count_of_easily_identifiable_digits(testdata)

26

In [95]:
get_count_of_easily_identifiable_digits(data)

330

## Part 2
Identify all digits, not just the easy ones.

Most wires are identifiable by their count, except a few.
The digit `4` is our key here, it helps differentiating between those.

In [96]:
def segments_to_digit(decoded_signal):
    segment_mapping = {
        frozenset("abcefg"): 0,
        frozenset("cf"): 1,
        frozenset("acdeg"): 2, 
        frozenset("acdfg"): 3,
        frozenset("bcdf"): 4,
        frozenset("abdfg"): 5,
        frozenset("abdefg"): 6,
        frozenset("acf"): 7,
        frozenset("abcdefg"): 8, 
        frozenset("abcdfg"): 9
    }
    
    return segment_mapping[frozenset(decoded_signal)]

In [97]:
def decode_signal(mapping, signal):
    return "".join([mapping[char] for char in signal])

In [98]:
def get_sum_of_output_digits(input_data):
    all_output_digits = []
    
    # We already know those because they only occur so much.
    fixes_occurrence_map = {
        6: "b",
        4: "e",
        9: "f", 
        # 7: "dg",  # Not unique! 
        # 8: "ac",  # Not unique!
    }
    
    for input_pattern, output_pattern in parse_input(input_data):
        decode_map = {}
        wirecount = defaultdict(int)
        four_signal = ""
        
        # Count the wires and look for a signal with length four
        # which just so happens to be unique and map to the digit four.
        for signal in input_pattern:
            for wire in signal:
                wirecount[wire] += 1
            if len(signal) == 4:
                four_signal = signal
        
        for wire, count in wirecount.items():
            # Map the fixed segments
            if count in fixes_occurrence_map:
                decode_map[wire] = fixes_occurrence_map[count]
            
            # Differentiate wire "d" and "g" which both occur 7 times.
            # One of which is used in the digit four, the other not.
            elif count == 7:
                if wire in four_signal:
                    decode_map[wire] = "d"
                else:
                    decode_map[wire] = "g"
            
            # Differentiate wire "c" and "a" which both occur 8 times.
            # One of which is used in the digit four, the other not.
            elif count == 8:
                if wire in four_signal:
                    decode_map[wire] = "c"
                else:
                    decode_map[wire] = "a"
        
        # Build the output digit from the now ready wire mapping
        output_digit = ""
        for signal in output_pattern:
            decoded_signal = decode_signal(decode_map, signal)
            digit = segments_to_digit(decoded_signal)
            output_digit += str(digit)

        all_output_digits.append(int(output_digit))
       
    return sum(all_output_digits)
        

In [99]:
get_sum_of_output_digits(testdata)

61229

In [100]:
get_sum_of_output_digits(data)

1010472